Query from fake bank database stored in [data/db/fake_bank.db](data/db/fake_bank.db)

In [1]:
import os
from dotenv import load_dotenv

# ibm genai
from genai.credentials import Credentials
from genai.model import Model
from genai.schemas import GenerateParams, ModelType
from genai.extensions.langchain import LangChainInterface  # langchain support

# langchain
from langchain.chains import SQLDatabaseChain, SQLDatabaseSequentialChain
from langchain.sql_database import SQLDatabase

load_dotenv()
api_key = os.getenv("GENAI_KEY", None)
api_url = os.getenv("GENAI_API", None)
creds = Credentials(api_key, api_endpoint=api_url)

In [2]:
# load model

llm = LangChainInterface(
        model=ModelType.FLAN_T5_11B,  # choice of model is not necessarily optimal
        credentials=Credentials(os.environ.get("GENAI_KEY")),
        params=GenerateParams(
            decoding_method="greedy",
            max_new_tokens=300,
            min_new_tokens=1,
            repetition_penalty=2,
        ).dict()
    )

In [3]:
# database connection and chain
db = SQLDatabase.from_uri("sqlite:///./data/db/fake_bank.db")
db_chain = SQLDatabaseSequentialChain.from_llm(llm, db, verbose=True)

In [4]:
db_chain.run('how many unique client names are in the bank')



> Entering new  chain...


/home/timw/venv/llm/lib/python3.11/site-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Table names to use:
['accounts', 'clients']

> Entering new  chain...
how many unique client names are in the bank
SQLQuery:SELECT count(DISTINCT Client_Name) FROM clients
SQLResult: [(6,)]
Answer:There are 6 unique client names in the bank.
> Finished chain.

> Finished chain.


'There are 6 unique client names in the bank.'

In [9]:
db_chain.run('how many accounts are in the deposit product family in the bank')



> Entering new  chain...
Table names to use:
['accounts', 'clients', 'transactions']

> Entering new  chain...
how many accounts are in the deposit product family  in the bank
SQLQuery:SELECT count(*) FROM accounts WHERE Product_Family = "deposit"
SQLResult: [(11,)]
Answer:11
> Finished chain.

> Finished chain.


'11'

In [11]:
db_chain.run('sum the dollars in the deposit product family accounts')



> Entering new  chain...
Table names to use:
['accounts', 'clients', 'transactions']

> Entering new  chain...
sum the dollars in the deposit product family accounts
SQLQuery:SELECT sum(Balance) FROM accounts WHERE Product_Family = "deposit"
SQLResult: [(20197586.46,)]
Answer:[20197586.46]
> Finished chain.

> Finished chain.


'[20197586.46]'

In [17]:
db_chain.run('sum the balance and group by client where product family is deposit')



> Entering new  chain...
Table names to use:
['transactions']

> Entering new  chain...
sum the balance and group by client where product family is deposit
SQLQuery:SELECT sum(Amount) FROM transactions WHERE Product_family = "deposit" GROUP BY Client_key

OperationalError: (sqlite3.OperationalError) no such column: Product_family
[SQL: SELECT sum(Amount) FROM transactions WHERE Product_family = "deposit" GROUP BY Client_key]
(Background on this error at: https://sqlalche.me/e/20/e3q8)